## Looking for world population?
 - The last data from the World Bank is the global county population in 2016
 - You can find the top countries of populations
 - You also can find it in the world map.
 ---

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from ipywidgets import interactive
%matplotlib inline

In [2]:
def plot_func(a, f):
    plt.figure(2)
    x = np.linspace(0, 2*np.pi, num=1000)
    y = a*np.sin(1/f*x)
    plt.plot(x,y)
    plt.ylim(-1.1, 1.1)
    plt.title('a sin(f)')
    plt.show()

interactive_plot = interactive(plot_func, a=(-1,0,0.1), f=(0.1, 1))
output = interactive_plot.children[-1]
output.layout.height = '300px'
interactive_plot

interactive(children=(FloatSlider(value=-1.0, description='a', max=0.0, min=-1.0), FloatSlider(value=0.55, des…

In [3]:
# read in population data from internet
import pandas as pd 
import datapackage

data_url = 'https://datahub.io/JohnSnowLabs/population-figures-by-country/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        df = pd.read_csv(resource.descriptor['path'])
df.set_index("Country_Code", inplace = True) 


In [4]:
# df.head()
# print(df.shape)

# the most recent year: 2016
df16= df.iloc[:, -1]
# df16.sort_values().tail(20)


url = "https://gist.githubusercontent.com/tadast/8827699/raw/3cd639fa34eec5067080a61c69e3ae25e3076abb/countries_codes_and_coordinates.csv"
df_code3 = pd.read_csv(url)
code3 = df_code3["Alpha-3 code"][0]

# for i, c in enumerate(code3):
#     print(i,c)
# # the length is not 3, and the longtitude/latitude have the same problem. 

# reset to 
df_code3["Code3"] = df_code3["Alpha-3 code"].map(lambda x: x.strip()[1:-1] )
df_code3["Lat"] = df_code3["Latitude (average)"].map(lambda x: x.strip()[1:-1]) 
df_code3["Long"] = df_code3["Longitude (average)"].map(lambda x: x.strip()[1:-1]) 

# set index 
df_code3.set_index("Code3", inplace=True )

# keep only the long and lat cols.
df_code3=df_code3.iloc[:, [-2, -1]]
# df_code3.shape # 256 rows


In [5]:
# there are no missing value, but some duplicated: 24 rows
df_code3.isna().sum() # none

# du=df_code3[df_code3.duplicated()] # 24 : 12 are extra
# print(du.size)

#remove
df_code3 = df_code3.drop_duplicates()
# df_code3

In [6]:
# use left-join-right
df_m= df_code3.join(df) 
# df_m

In [7]:
# some country's name is missing: 32 countries
df_m.Country.isna().sum()

# # keep only the country which has name in it 
df_good = df_m[df_m.Country.notna()]

# df_good

In [8]:
# set-index by Country name
df_good.set_index(["Country"], inplace=True)

# no duplicated
# df_good[df_good.duplicated()].sum().sum()

In [9]:
# # Population top 10 in 2016
df_2016 =pd.DataFrame(df_good.loc[:, 'Year_2016'])
df_2016.dropna(inplace=True)
df_2016.sort_values(by="Year_2016", ascending =False).head(10).applymap(lambda x:int(x)) 

,Year_2016
Country,
China,1378665000
India,1324171354
United States,323127513
Indonesia,261115456
Brazil,207652865
Pakistan,193203476
Nigeria,185989640
Bangladesh,162951560
Russian Federation,144342396
